In [1]:
# Install library
!pip install scikit-image opencv-python torch torchvision ffmpeg-python scikit-video

In [2]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!apt-get install p7zip-full -y

Looking in indexes: https://download.pytorch.org/whl/cu118
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [3]:
# import
import os, sys, cv2, glob, torch, numpy as np
from google.colab import drive
from skimage.metrics import structural_similarity as ssim_metric
from PIL import Image
#import skvideo.io

In [4]:
# RIFE
!git clone https://github.com/hzwer/arXiv2020-RIFE

fatal: destination path 'arXiv2020-RIFE' already exists and is not an empty directory.


In [5]:
!gdown --id 1APIzVeI-4ZZCEuIRE1m6WYfSCaOsi_7_
!7z e RIFE_trained_model_v3.6.zip

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1APIzVeI-4ZZCEuIRE1m6WYfSCaOsi_7_
To: /content/RIFE_trained_model_v3.6.zip
100% 11.3M/11.3M [00:00<00:00, 17.0MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 11332064 bytes (11 MiB)

Extracting archive: RIFE_trained_model_v3.6.zip
--
Path = RIFE_trained_model_v3.6.zip
Type = zip
Physical Size = 11332064

  0%    
Would you like to replace the existing file:
  Path:     ./.DS_Store
  Size:     6148 bytes (7 KiB)
  Modified: 2021-06-17 11:33:40
with the file from archive:
  Path:    

In [6]:
!mkdir /content/arXiv2020-RIFE/train_log
%cd /content/arXiv2020-RIFE/train_log
%cd /content/arXiv2020-RIFE/
!gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
!pip install git+https://github.com/rk-exxec/scikit-video.git@numpy_deprecation

mkdir: cannot create directory ‘/content/arXiv2020-RIFE/train_log’: File exists
/content/arXiv2020-RIFE/train_log
/content/arXiv2020-RIFE
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
To: /content/arXiv2020-RIFE/demo.mp4
100% 54.6M/54.6M [00:00<00:00, 109MB/s]
  Cloning https://github.com/rk-exxec/scikit-video.git (to revision numpy_deprecation) to /tmp/pip-req-build-mvan56zj
  Running command git clone --filter=blob:none --quiet https://github.com/rk-exxec/scikit-video.git /tmp/pip-req-build-mvan56zj
  Running command git checkout -b numpy_deprecation --track origin/numpy_deprecation
  Switched to a new branch 'numpy_deprecation'
  Branch 'numpy_deprecation' set up to track remote branch 'numpy_deprecation' from 'origin'.
  Reso

In [7]:
drive.mount('/content/drive')
!ls /content/drive/MyDrive/project


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ff.jpg	fl.jpg	fm.jpg	man_drinking_water.mp4


In [8]:
!ls /content/drive/MyDrive/project

ff.jpg	fl.jpg	fm.jpg	man_drinking_water.mp4


In [9]:
video = '/content/drive/MyDrive/project/man_drinking_water.mp4'

In [10]:
# Make output folder

output_folder = '/content/drive/MyDrive/project'
os.makedirs(output_folder, exist_ok=True)


In [11]:
# Frame calculation

cap = cv2.VideoCapture(video)
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"FPS         : {fps} \nTotal Frames: {total_frame}")

FPS         : 25 
Total Frames: 868


In [12]:
# Extract ff,fm,fl
import cv2, os

def save_frame(position, name):
    cap.set(cv2.CAP_PROP_POS_FRAMES, position)
    ret, frame = cap.read()
    if ret:
        path = os.path.join(output_folder, name)
        cv2.imwrite(path, frame)
        print(f"{name} saved: {path}")
        return path
    else:
        print(f"❌ Failed to read frame at position {position}")
    return None

# Use min(total_frame-2, total_frame-1) to stay within valid range
first_frame_path = save_frame(0, "ff.jpg")
middle_frame_path = save_frame(total_frame // 2, "fm.jpg")
last_frame_path = save_frame(-1, "fl.jpg")  # <- safer for end of video


ff.jpg saved: /content/drive/MyDrive/project/ff.jpg
fm.jpg saved: /content/drive/MyDrive/project/fm.jpg
fl.jpg saved: /content/drive/MyDrive/project/fl.jpg


In [13]:
# load frame
ff = cv2.imread(first_frame_path)
fl = cv2.imread(last_frame_path)
fm = cv2.imread(middle_frame_path)

In [14]:
# match resolution
def resize32(img):
    hight, width = img.shape[:2]
    hight_new, width_new = (hight // 32) * 32, (width // 32) * 32
    return cv2.resize(img, (width_new, hight_new))
ff = resize32(ff)
fl = resize32(fl)

In [15]:
# np array to tensor
first_tensor = torch.from_numpy(ff.transpose(2,0,1)).unsqueeze(0).float().cuda() / 255.0
last_tensor  = torch.from_numpy(fl.transpose(2,0,1)).unsqueeze(0).float().cuda() / 255.0

In [45]:
# load RIFE model
!python3 inference_video.py --exp=1 --video=/content/arXiv2020-RIFE/man_drinking_water.mp4

Loaded v3.x HD model.
/content/arXiv2020-RIFE/man_drinking_water.mp4, 868.0 frames in total, 25.0FPS to 50.0FPS
The audio will be merged after interpolation process
100% 866/868.0 [02:19<00:00,  6.22it/s]
error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c

In [50]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [57]:
print(first_frame_path, os.path.exists(first_frame_path))
print(last_frame_path, os.path.exists(last_frame_path))
print(middle_frame_path, os.path.exists(middle_frame_path))


/content/drive/MyDrive/project/ff.jpg True
/content/drive/MyDrive/project/fl.jpg True
/content/drive/MyDrive/project/fm.jpg True


In [58]:
import cv2

# Read images as arrays (not paths)
first_frame = cv2.imread(first_frame_path)   # shape (H,W,3)
last_frame  = cv2.imread(last_frame_path)
middle_frame = cv2.imread(middle_frame_path)

# Check shapes
print(first_frame.shape, last_frame.shape, middle_frame.shape)


(720, 1280, 3) (720, 1280, 3) (720, 1280, 3)


In [59]:
import torch
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Convert to tensors
first_tensor = torch.from_numpy(first_frame.transpose(2,0,1)).unsqueeze(0).float() / 255.0
last_tensor  = torch.from_numpy(last_frame.transpose(2,0,1)).unsqueeze(0).float() / 255.0

# Move to device
first_tensor = first_tensor.to(device)
last_tensor  = last_tensor.to(device)


Using device: cuda


In [60]:
from skimage.metrics import structural_similarity as ssim_metric

def compute_ssim(gen, ref):
    gen = cv2.resize(gen, (ref.shape[1], ref.shape[0]))
    return ssim_metric(cv2.cvtColor(gen, cv2.COLOR_BGR2GRAY),
                       cv2.cvtColor(ref, cv2.COLOR_BGR2GRAY))


In [63]:
import torch
import sys
import os

# Add RIFE code folder to path
sys.path.append('/content/arXiv2020-RIFE')

from train_log.RIFE_HDv3 import Model  # or use RIFE_HDv3 if you have it

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize and load model
model = Model()
model.load_model('./train_log', -1)  # replace './train_log' with your model weights path
model.eval()
model.device()  # set model to device
print("RIFE model loaded successfully!")


Using device: cuda
RIFE model loaded successfully!


In [65]:
# generate middle frame until ssim>90%
def compute_ssim(gen, ref):
    gen = cv2.resize(gen, (ref.shape[1], ref.shape[0]))
    return ssim_metric(cv2.cvtColor(gen, cv2.COLOR_BGR2GRAY),
                       cv2.cvtColor(ref, cv2.COLOR_BGR2GRAY))

similarity = 0
threshold = 0.9
iteration = 0
max_iterations = 10


ssim_folder = os.path.join(output_folder, "ssim_iterations")
os.makedirs(ssim_folder, exist_ok=True)



while similarity < threshold and iteration < max_iterations:
    iteration += 1
    with torch.no_grad():
        middle_tensor = model.inference(first_tensor, last_tensor)
    fg = (middle_tensor[0].cpu().numpy().transpose(1,2,0) * 255.0).astype(np.uint8)
    similarity = compute_ssim(fg, fm)
    print(f"Iteration {iteration}: SSIM = {similarity:.4f}")

    iter_path = os.path.join(ssim_folder, f"ssim_iter_{iteration:02d}.jpg")
    cv2.imwrite(iter_path, fg)

Iteration 1: SSIM = 0.4587
Iteration 2: SSIM = 0.4587
Iteration 3: SSIM = 0.4587
Iteration 4: SSIM = 0.4587
Iteration 5: SSIM = 0.4587
Iteration 6: SSIM = 0.4587
Iteration 7: SSIM = 0.4587
Iteration 8: SSIM = 0.4587
Iteration 9: SSIM = 0.4587
Iteration 10: SSIM = 0.4587


In [66]:
# save frame
if similarity >= threshold:
    fg_frame_path = os.path.join(output_folder, 'fg.jpg')
    cv2.imwrite(fg_frame_path, fg)
    print(f"SSIM > 90% frame saved at: {fg_frame_path}")
else:
    print(f"SSIM threshold not reached. Final = {similarity:.4f}")

SSIM threshold not reached. Final = 0.4587


In [67]:
# generate middle frame until color accuracy<90%
def color_similarity(gen, ref):
    gen_lab = cv2.cvtColor(gen, cv2.COLOR_BGR2LAB).astype("float32")
    ref_lab = cv2.cvtColor(ref, cv2.COLOR_BGR2LAB).astype("float32")
    delta_e = np.sqrt(np.mean((gen_lab - ref_lab) ** 2))
    return max(0, 1 - (delta_e / 100))

threshold = 0.9
max_iterations = 20
iteration = 0
similarity = color_similarity(fg, fm)
print(f"Initial color similarity: {similarity:.4f}")



color_folder = os.path.join(output_folder, "color_iterations")
os.makedirs(color_folder, exist_ok=True)

while similarity < threshold and iteration < max_iterations:
    iteration += 1
    alpha = 0.1
    fg = cv2.addWeighted(fg, 1 - alpha, fm, alpha, 0)
    similarity = color_similarity(fg, fm)
    print(f"Iteration {iteration}: Color similarity = {similarity:.4f}")

    iter_path = os.path.join(color_folder, f"color_iter_{iteration:02d}.jpg")
    cv2.imwrite(iter_path, fg)

Initial color similarity: 0.7378
Iteration 1: Color similarity = 0.7642
Iteration 2: Color similarity = 0.7879
Iteration 3: Color similarity = 0.8093
Iteration 4: Color similarity = 0.8284
Iteration 5: Color similarity = 0.8455
Iteration 6: Color similarity = 0.8609
Iteration 7: Color similarity = 0.8747
Iteration 8: Color similarity = 0.8871
Iteration 9: Color similarity = 0.8982
Iteration 10: Color similarity = 0.9081


In [68]:
# save frame
fg_color_frame_path = os.path.join(output_folder, 'fgc.jpg')
cv2.imwrite(fg_color_frame_path, fg)
if similarity >= threshold:
    print(f"Color similarity threshold reached. Refined frame saved at: {fg_color_frame_path}")
else:
    print(f"Threshold not reached. Final color similarity = {similarity:.4f}")

Color similarity threshold reached. Refined frame saved at: /content/drive/MyDrive/project/fgc.jpg


In [69]:
# visual representation ssim
ssim_images = sorted(glob.glob(os.path.join(ssim_folder, "*.jpg")))

if ssim_images:
    frames = [Image.open(img) for img in ssim_images]
    ssim_gif_path = os.path.join(output_folder, "ssim_progress.gif")
    frames[0].save(
        ssim_gif_path,
        save_all=True,
        append_images=frames[1:],
        duration=500,
        loop=0
    )
    print(f"SSIM progress GIF saved at: {ssim_gif_path}")
else:
    print("No SSIM iteration images found!")


SSIM progress GIF saved at: /content/drive/MyDrive/project/ssim_progress.gif


In [70]:
# visual representation color
color_images = sorted(glob.glob(os.path.join(color_folder, "*.jpg")))

if color_images:
    frames = [Image.open(img) for img in color_images]
    color_gif_path = os.path.join(output_folder, "color_progress.gif")
    frames[0].save(
        color_gif_path,
        save_all=True,
        append_images=frames[1:],
        duration=500,
        loop=0
    )
    print(f"Color progress GIF saved at: {color_gif_path}")
else:
    print("No color iteration images found!")

Color progress GIF saved at: /content/drive/MyDrive/project/color_progress.gif


In [71]:
# Free resource
cap.release()
print(f"All frames saved in: {output_folder}")

All frames saved in: /content/drive/MyDrive/project
